In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vishpand\\OneDrive - Nokia\\1) Nokia Projects\\Projects\\e2e-machine-learning-project-with-mlflow\\notebooks'

In [3]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\vishpand\\OneDrive - Nokia\\1) Nokia Projects\\Projects\\e2e-machine-learning-project-with-mlflow'

In [6]:
import pandas as pd
df = pd.read_csv("./artifacts/data_ingestion/winequality-red.csv")

In [8]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [9]:
temp = dict(zip(df.columns, df.iloc[1,:]))

In [13]:
for k, v in temp.items():
    print(f"{k}: float")

fixed acidity: float
volatile acidity: float
citric acid: float
residual sugar: float
chlorides: float
free sulfur dioxide: float
total sulfur dioxide: float
density: float
pH: float
sulphates: float
alcohol: float
quality: float


In [19]:
info_dict = {str(column): str(dtype) for column, dtype in zip(df.columns, df.dtypes)}

In [20]:
print(info_dict)

{'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}


In [21]:
for k, v in info_dict.items():
    print(f"{k}: {v}")

fixed acidity: float64
volatile acidity: float64
citric acid: float64
residual sugar: float64
chlorides: float64
free sulfur dioxide: float64
total sulfur dioxide: float64
density: float64
pH: float64
sulphates: float64
alcohol: float64
quality: int64


In [22]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [29]:
from pydantic import BaseModel
from pathlib import Path

class DataValidationConfig(BaseModel):
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

    model_config = {
        "frozen": True
    }

In [25]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config

In [34]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation status: {validation_status}")

            # print(type(validation_status))
            if type(validation_status) != bool:
                validation_status = False

            return validation_status
        
        except Exception as e:
            raise e
        

In [35]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-05-21 00:16:44,554: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 00:16:44,557: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 00:16:44,560: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-21 00:16:44,563: INFO: common: created directory at: artifacts]
[2024-05-21 00:16:44,564: INFO: common: created directory at: artifacts/data_validation]
<class 'bool'>
